In [30]:
# 测试一下
import numpy as np
import sympy as sp

def op(A, x, b):
    """
    Compute x = A^T * H(A*x - b)
    
    Parameters:
    -----------
    A : numpy.ndarray
        3x3 matrix
    x : numpy.ndarray
        Input vector
    b : numpy.ndarray
        Vector of the same dimension as A*x
    
    Returns:
    --------
    numpy.ndarray
        Result of A^T * H(A*x - b)
    """

    try_to_add = A * x
    overflow = try_to_add - b

    overflow = (try_to_add - b).applyfunc(lambda x: sp.Max(x, 0))
    
    # Compute A^T * H(A*x - c)
    compensate = A.T * overflow
    
    return try_to_add - compensate

x = sp.Matrix([[4, 9, 0]]).T
b = sp.Matrix([[4, 9, 9]]).T
A = sp.Matrix([[0, 0, 0], [0, 1, 0], [1, 0, 1]])
print(A.pinv() * A)
x_new = op(A, x, b)
print(x_new)


Matrix([[1/2, 0, 1/2], [0, 1, 0], [1/2, 0, 1/2]])
Matrix([[0], [9], [4]])


In [16]:
A_new = np.array([[1, 0, 0], [0, 0, 0], [0, 1, 1]])
x_new_new = op(A_new, x_new, b)
print(x_new_new)

[ 0. -4.  9.]


In [38]:
A = sp.zeros(3, 3)
A[0, 0] = -1
A[2, 0] = 1
print(A)

Matrix([[-1, 0, 0], [0, 0, 0], [1, 0, 0]])


In [46]:
def delta_op(i, j, size = 3):
    """
    计算增量
    [
        -1, 0, 0,
        0, 0, 0,
        1, 0, 0
    ]
    相当于:
      d a_i = - a_i
      d a_j = a_i
    """
    
    delta = sp.zeros(size, size)
    delta[i, i] = -1
    delta[j, i] = 1
    
    return delta

print(sp.eye(3, 3) + delta_op(0, 2))

Matrix([[0, 0, 0], [0, 1, 0], [1, 0, 1]])


In [48]:
def add_op(i, j, x, b, size = 3):
    try_op = sp.eye(size, size) + delta_op(i, j, size)
    try_add = try_op * x

    overflow = try_add - b
    overflow_guard = overflow.applyfunc(lambda x: sp.Max(x, 0))

    compensate = delta_op(j, i, size) * overflow_guard

    return try_add + compensate

x = sp.Matrix([[4, 9, 0]]).T
b = sp.Matrix([[4, 9, 9]]).T
x_new = add_op(0, 2, x, b)
print(x_new)
x_new_new = add_op(1, 2, x_new, b)
print(x_new_new)

Matrix([[0], [9], [4]])
Matrix([[0], [4], [9]])
